In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2022-10-10T11:00:37.724767-04:00

Python implementation: CPython
Python version       : 3.8.9
IPython version      : 8.4.0

Compiler    : Clang 13.1.6 (clang-1316.0.21.2.5)
OS          : Darwin
Release     : 21.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 20
Architecture: 64bit



In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import KFold
import xgboost as xgb



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../input/xgval_2.csv
../input/test.csv
../input/xgval_0.csv
../input/xgval_1.csv
../input/xgtrain_1.csv
../input/xgtrain_0.csv
../input/xgtrain_2.csv
../input/train.csv
../input/sample_submission.csv


In [4]:
xgb.__version__

'1.6.2'

In [5]:
%watermark --iversions

numpy  : 1.23.1
pandas : 1.4.3
xgboost: 1.6.2



In [6]:
def CRMSLE(truth, pred): 
    rmsle1 = (mean_squared_log_error(truth[:,0], pred[:,0]))**(1/2) 
    rmsle2 = (mean_squared_log_error(truth[:,1], pred[:,1]))**(1/2) 
    rmsle3 = (mean_squared_log_error(truth[:,2], pred[:,2]))**(1/2) 
    return (rmsle1 + rmsle2 + rmsle3)/3

In [7]:
sample_submission = pd.read_csv('../input/sample_submission.csv')
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [8]:
train.head()

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
0,2010-03-10 18:00:00,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,2.5,12.0,167.7
1,2010-03-10 19:00:00,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,2.1,9.9,98.9
2,2010-03-10 20:00:00,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,2.2,9.2,127.1
3,2010-03-10 21:00:00,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2,2.2,9.7,177.2
4,2010-03-10 22:00:00,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2,1.5,6.4,121.8


In [9]:
test.head()

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5
0,2011-01-01 00:00:00,8.0,41.3,0.4375,1108.8,745.7,797.1,880.0,1273.1
1,2011-01-01 01:00:00,5.1,51.7,0.4564,1249.5,864.9,687.9,972.8,1714.0
2,2011-01-01 02:00:00,5.8,51.5,0.4689,1102.6,878.0,693.7,941.9,1300.8
3,2011-01-01 03:00:00,5.0,52.3,0.4693,1139.7,916.2,725.6,1011.0,1283.0
4,2011-01-01 04:00:00,4.5,57.5,0.4650,1022.4,838.5,871.5,967.0,1142.3


In [11]:
columns = test.columns[1:]
columns

Index(['deg_C', 'relative_humidity', 'absolute_humidity', 'sensor_1',
       'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5'],
      dtype='object')

In [13]:
X = train[columns].values
X_test = test[columns].values
target = train[['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']].values

In [15]:
target

array([[  2.5,  12. , 167.7],
       [  2.1,   9.9,  98.9],
       [  2.2,   9.2, 127.1],
       ...,
       [  1.6,   5.2, 227.4],
       [  1.5,   4.6, 199.8],
       [  1.4,   4.1, 186.5]])

In [18]:
%%time
train_oof = np.zeros(target.shape)
kf = KFold(n_splits=5, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_x = X[train_index]
    val_x = X[val_index]
    train_target = target[train_index]
    val_target = target[val_index]
    model = xgb.XGBRegressor(tree_method='hist')
    model.fit(train_x, train_target)
    val_preds = np.clip(model.predict(val_x), 0, None)
    train_oof[val_index] = val_preds
    print(CRMSLE(val_target, val_preds))

Fitting fold 1
0.18776819586578797
Fitting fold 2
0.1929879621402617
Fitting fold 3
0.2190537799131055
Fitting fold 4
0.20838318477897158
Fitting fold 5
0.20809073420227953
CPU times: user 48.9 s, sys: 55.3 s, total: 1min 44s
Wall time: 21.1 s


In [19]:
print(CRMSLE(target, train_oof))

0.20368602402793204
